In [ ]:
import random
import numpy as np
from IO.imgIO import *
from IO.segIO import *
epochs=5
batch_size=10
lr=0.1
val_percent=0.05
save_cp=True
gpu=False
img_scale=0.5
    
dir_data = './CT_data_batch1/'
dir_img = '/DICOM_anon/'
dir_out = '/Ground/'
dir_checkpoint = './checkpoints/'

x = get_training_dataset(dir_data, dir_img, dir_out)
y = dataset_to_array(x,[80,512,512])
train_set,val_set = prep_for_training2d(y,nslice = 5)

In [ ]:
import torch.optim as optim
import torch
from DL.unet import *
net = UNet1(n_channels=5,o_channels=1)
epochs=5
batch_size=1

print('''
    Starting training:
    Epochs: {}
    Batch size: {}
    Learning rate: {}
    Training size: {}
    Validation size: {}
    Checkpoints: {}
    CUDA: {}
'''.format(epochs, batch_size, lr, len(train_set['in_set']),
           len(val_set['in_set']), str(save_cp), str(gpu)))

# shuffle, batch segment
# network definition
# training
N_train = len(train_set['in_set'])

optimizer = optim.Adam(net.parameters(),
                      lr=lr)

criterion = nn.BCELoss()

train_input_img = train_set['in_set']
train_output_img = train_set['out_set']
val_input_img = val_set['in_set']
val_output_img = val_set['out_set']
comb = list(zip(train_input_img, train_output_img))

for epoch in range(epochs):
    print('Starting epoch {}/{}.'.format(epoch + 1, epochs))
    net.train()
    random.shuffle(comb)

    i_imgs, o_imgs = zip(*comb)

    epoch_loss = 0


    for i in range(len(i_imgs)//batch_size):
        imgs = np.array([i_imgs[i] for i in range(i*batch_size,min([(i+1)*batch_size,len(i_imgs)]))]).astype(np.float32)
        true_masks = np.array([o_imgs[i] for i in range(i*batch_size,min([(i+1)*batch_size,len(i_imgs)]))]).astype(np.float32)
        
        imgs = Variable(torch.from_numpy(imgs))
        true_masks = Variable(torch.from_numpy(true_masks))
        if gpu:
            imgs = imgs.cuda()
            true_masks = true_masks.cuda()
        optimizer.zero_grad()
        masks_pred = net(imgs)
        masks_probs_flat = masks_pred.view(-1)

        true_masks_flat = true_masks.view(-1)

        loss = criterion(masks_probs_flat, true_masks_flat)
        epoch_loss += loss.data.numpy()
        
        # loss.item()
        print('{0:.4f} --- loss: {1:.6f}'.format(i * batch_size / N_train, epoch_loss[0]))

        
        loss.backward()
        optimizer.step()

    print('Epoch finished ! Loss: {}'.format(epoch_loss / i))

    if 0:
        val_dice = eval_net(net, val, gpu)
        print('Validation Dice Coeff: {}'.format(val_dice))

    if save_cp:
        torch.save(net.state_dict(),
                   dir_checkpoint + 'CP{}.pth'.format(epoch + 1))
        print('Checkpoint {} saved !'.format(epoch + 1))